# Preprocessing cho HOG
## Chuẩn bị ảnh grayscale cho HOG feature extraction

Các bước thực hiện:
1. **Load dữ liệu**: Đọc file từ thư mục split_data
2. **Preprocessing**: Resize + chuyển sang grayscale
3. **Lưu ảnh**: Lưu ảnh grayscale đã preprocess

**Lưu ý**: Bước này chỉ preprocessing, chưa trích xuất HOG features!


In [1]:
# 1. Import thư viện
import os
import json
import cv2
import numpy as np
from tqdm import tqdm


In [ ]:
# 2. Load class mapping
with open('split_data/class_mapping.json', 'r') as f:
    class_mapping = json.load(f)

# Chuyển key từ string sang int
class_mapping = {int(k): v for k, v in class_mapping.items()}

print(f"Số lớp: {len(class_mapping)}")
print(f"Classes: {list(class_mapping.values())}")


Số lớp: 23
Classes: ['Clams', 'Corals', 'Crabs', 'Dolphin', 'Eel', 'Fish', 'Jelly Fish', 'Lobster', 'Nudibranchs', 'Octopus', 'Otter', 'Penguin', 'Puffers', 'Sea Rays', 'Sea Urchins', 'Seahorse', 'Seal', 'Sharks', 'Shrimp', 'Squid', 'Starfish', 'Turtle_Tortoise', 'Whale']


In [3]:
# 3. Định nghĩa hàm preprocessing
IMAGE_SIZE = (128, 128)
# Preprocessing ảnh cho HOG: resize + grayscale
def preprocess_for_hog(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None
    
    # Resize
    img = cv2.resize(img, IMAGE_SIZE)
    
    # Chuyển sang grayscale
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Normalize 0-1
    img_gray = img_gray.astype('float32') / 255.0
    
    return img_gray

print(f"✓ Image size: {IMAGE_SIZE}")
print(f"✓ Method: Grayscale preprocessing for HOG")


✓ Image size: (128, 128)
✓ Method: Grayscale preprocessing for HOG


In [4]:
# 4. Load và preprocessing train data

train_dir = 'split_data/train'
X_train = []
y_train = []

for class_idx, class_name in class_mapping.items():
    class_path = os.path.join(train_dir, class_name)
    if not os.path.exists(class_path):
        continue
    
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    
    for img_file in tqdm(images, desc=f"Train {class_name}"):
        img_path = os.path.join(class_path, img_file)
        img = preprocess_for_hog(img_path)
        
        if img is not None:
            X_train.append(img)
            y_train.append(class_idx)

X_train = np.array(X_train)
y_train = np.array(y_train)

print(f"Train shape: {X_train.shape}")


Train Whale: 100%|██████████| 458/458 [00:00<00:00, 1628.14it/s]


Train shape: (10968, 128, 128)


In [5]:
# 5. Load và preprocessing test data

test_dir = 'split_data/test'
X_test = []
y_test = []

for class_idx, class_name in class_mapping.items():
    class_path = os.path.join(test_dir, class_name)
    if not os.path.exists(class_path):
        continue
    
    images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif'))]
    
    for img_file in tqdm(images, desc=f"Test {class_name}"):
        img_path = os.path.join(class_path, img_file)
        img = preprocess_for_hog(img_path)
        
        if img is not None:
            X_test.append(img)
            y_test.append(class_idx)

X_test = np.array(X_test)
y_test = np.array(y_test)

print(f"Test shape: {X_test.shape}")


Test Whale: 100%|██████████| 114/114 [00:00<00:00, 1624.90it/s]

Test shape: (2743, 128, 128)


In [6]:
# 6. Lưu ảnh đã preprocessing

output_dir = 'preprocessed_hog'
os.makedirs(output_dir, exist_ok=True)

# Lưu train data
np.savez_compressed(
    f'{output_dir}/train_data.npz',
    X=X_train,
    y=y_train
)

# Lưu test data
np.savez_compressed(
    f'{output_dir}/test_data.npz',
    X=X_test,
    y=y_test
)

# Lưu class mapping
with open(f'{output_dir}/class_mapping.json', 'w') as f:
    json.dump(class_mapping, f, indent=2, ensure_ascii=False)

# Lưu preprocessing info
preprocessing_info = {
    'method': 'Grayscale Preprocessing',
    'image_size': IMAGE_SIZE,
    'color_space': 'Grayscale',
    'normalization': '0-1',
    'train_samples': len(X_train),
    'test_samples': len(X_test),
    'num_classes': len(class_mapping)
}

with open(f'{output_dir}/preprocessing_info.json', 'w') as f:
    json.dump(preprocessing_info, f, indent=2)

print("Đã lưu train_data.npz, test_data.npz và metadata")


Đã lưu train_data.npz, test_data.npz và metadata
